In [1]:
import pandas as pd

In [7]:
df_price = pd.read_csv("../../data/raw/movie_price_spreadsheets.csv", engine= "python", encoding='utf-8')
df_download = pd.read_csv("../../data/cleaned/movie_download_cleaned.csv", engine= "python", encoding='utf-8')
df_customer = pd.read_csv("../../data/cleaned/movie_customer_cleaned.csv", engine= "python", encoding='utf-8')

In [5]:
df_price.describe()

,contract_year,release_year,runtime,imdb_score,votes,reviews_users,reviews_critics,budget,income_usa,income_ww,theater_opening,theater_total,price_year,basic_price,dc_rate,subscribe_price,contract_price,studio_score,price_class
count,394.000000,394.000000,394.000000,394.000000,3.940000e+02,394.000000,394.000000,3.940000e+02,3.940000e+02,3.940000e+02,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000
mean,2014.756345,2012.497462,126.345178,7.183756,5.081580e+05,1254.784264,463.908629,1.484695e+08,2.995764e+08,7.708920e+08,3775.157360,3985.236041,2016.378173,1.020305,0.075508,0.943528,1220.604061,8.304569,5.017766
std,1.070789,3.551331,22.164608,0.842322,3.495355e+05,1007.944782,160.927581,6.913259e+07,1.141374e+08,3.705083e+08,870.286432,298.981260,1.342828,0.068750,0.038289,0.078432,91.745558,1.979187,4.463843
min,2014.000000,2005.000000,87.000000,4.100000,3.990000e+04,148.000000,151.000000,4.500000e+06,1.102127e+08,1.407958e+08,1.000000,3142.000000,2014.000000,0.900000,0.000000,0.810000,955.000000,6.000000,1.000000
25%,2014.000000,2009.000000,105.250000,6.700000,2.683822e+05,511.000000,345.000000,8.000000e+07,2.299287e+08,5.529826e+08,3702.000000,3796.000000,2015.000000,1.000000,0.050000,0.900000,1155.000000,6.000000,1.000000
50%,2014.000000,2013.000000,127.000000,7.300000,4.717730e+05,1001.000000,429.500000,1.500000e+08,2.925762e+08,7.450131e+08,3997.000000,4022.500000,2017.000000,1.000000,0.100000,0.900000,1299.000000,10.000000,1.000000
75%,2015.000000,2015.000000,144.000000,7.800000,6.465240e+05,1549.000000,579.000000,2.000000e+08,3.479118e+08,9.385804e+08,4207.000000,4207.000000,2018.000000,1.000000,0.100000,1.000000,1299.000000,10.000000,10.000000
max,2018.000000,2018.000000,169.000000,9.000000,2.134569e+06,6344.000000,837.000000,3.210000e+08,7.605076e+08,2.789968e+09,4529.000000,4535.000000,2018.000000,1.200000,0.100000,1.200000,1299.000000,10.000000,10.000000


In [12]:
# 연당 평균 다운로드 가격
df_pr_mean_year = pd.DataFrame([df_price['subscribe_price'].groupby(df_price['price_year']).mean()])
df_pr_mean_year

price_year,2014,2015,2016,2017,2018
subscribe_price,0.927222,0.958507,0.95128,0.941383,0.936887


In [59]:
# 누적 등록 회원 수
num_register = pd.DataFrame([df_customer['register_year'].value_counts()]).sort_index(axis=1)
num_register_cum = num_register.cumsum(axis = 1)
num_register_cum

,2014,2015,2016,2017,2018
register_year,1017,1745,2407,3268,3277


In [28]:
# 연당 탈퇴 회원 수
num_drop = pd.DataFrame([df_customer['drop_year'].value_counts()[1:]])
num_drop = num_drop.reindex(sorted(num_drop.columns), axis=1)
num_drop

,2015,2016,2017,2018
drop_year,130,143,189,197


In [60]:
# 연간 순 활동 회원 수(active)
num_custom_year = []
num_custom_year.append(int(num_register_cum[2014]))


num_custom_year.append(int(num_register_cum[2015]) - int(num_drop['2015']))
num_custom_year.append(int(num_register_cum[2016]) - int(num_drop['2016']))
num_custom_year.append(int(num_register_cum[2017]) - int(num_drop['2017']))
num_custom_year.append(int(num_register_cum[2018]) - int(num_drop['2018']))

num_custom_year = pd.DataFrame([num_custom_year], columns=num_register_cum.columns, index=['active_user']).sort_index(axis=1)
num_custom_year

,2014,2015,2016,2017,2018
active_user,1017,1615,2264,3079,3080


In [68]:
df_main = pd.DataFrame([df_download['down_year'].value_counts()], index=['down_cnt'])
df_main = df_main.reindex(sorted(df_main.columns), axis=1).transpose()

In [69]:
df_main['active_user'] = num_custom_year.transpose()

In [70]:
df_main['per_capita_down'] = df_main['down_cnt']/df_main['active_user']
df_main['price_mean'] = df_pr_mean_year.transpose()

In [73]:
d_price_mean = [0]
for i in range(2015,2019):
    d_price_mean.append(df_main['price_mean'][i] - df_main['price_mean'][i-1])
    
d_per_capita_down = [0]
for i in range(2015,2019):
    d_per_capita_down.append(df_main['per_capita_down'][i] - df_main['per_capita_down'][i-1])

df_main['d_price_mean'] = d_price_mean
df_main['d_per_capita_down'] = d_per_capita_down

In [82]:
elasticity = [0]
for i in range(2015,2019):
    elasticity.append((df_main['d_per_capita_down'][i] / df_main['per_capita_down'][i])/(df_main['d_price_mean'][i] / df_main['price_mean'][i]))
df_main['elasticity_all'] = elastcity

,down_cnt,active_user,per_capita_down,price_mean,d_price_mean,d_per_capita_down,elastcity_all,elasticity_all
2014,5031,1017,4.946903,0.927222,0.000000,0.000000,0.000000,0.000000
2015,11981,1615,7.418576,0.958507,0.031285,2.471673,10.207667,10.207667
2016,15063,2264,6.653269,0.951280,-0.007227,-0.765307,15.140940,15.140940
2017,21126,3079,6.861319,0.941383,-0.009898,0.208050,-2.884036,-2.884036
2018,19884,3080,6.455844,0.936887,-0.004496,-0.405474,13.087397,13.087397


In [85]:
df_main

,down_cnt,active_user,per_capita_down,price_mean,d_price_mean,d_per_capita_down,elasticity_all
2014,5031,1017,4.946903,0.927222,0.000000,0.000000,0.000000
2015,11981,1615,7.418576,0.958507,0.031285,2.471673,10.207667
2016,15063,2264,6.653269,0.951280,-0.007227,-0.765307,15.140940
2017,21126,3079,6.861319,0.941383,-0.009898,0.208050,-2.884036
2018,19884,3080,6.455844,0.936887,-0.004496,-0.405474,13.087397
